In [1]:
!pip install tqdm==4.42.1
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install netcdf4==1.5.3
!pip install xarray
!pip install numpy==1.18.1
!pip install rasterio
!pip install xgboost==0.90
!pip install joblib==0.14.1




/home/edward_brown/anaconda3/envs/plant/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import xarray as xr
import numpy as np
import re
import json
import rasterio
import pickle
import joblib
import xgboost as xgb
import pandas as pd
import os, sys
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import xgboost as xgb
from sklearn.metrics import r2_score
import random

/home/edward_brown/anaconda3/envs/plant/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# Add the locations of the data folders here, and the crop file locations
seasonal_data_folder = '../data/seasonal_era_agriclimatic/'
ten_day_data_folder = '../data/10-day-data/'
crop_data_2005_file = '../data/spam2005v3r2_global_yield/spam2005V3r2_global_Y_TA.csv'
crop_data_2010_file = '../data/spam2010v1r1_global_yield/spam2010V1r1_global_Y_TA.csv'

ten_day_feature_files = {
    'BEDD': 'BEDD_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'FD': 'FD_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'R20mm': 'R20mm_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'R10mm': 'R10mm_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'ID': 'ID_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TG': 'TG_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TN': 'TN_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'DTR': 'DTR_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'RR1': 'RR1_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'RR': 'RR_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'SDII': 'SDII_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'SU': 'SU_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TG': 'TG_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TNn': 'TNn_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TR': 'TR_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TX': 'TX_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TXn': 'TXn_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TXx': 'TXx_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc'
}

seasonal_feature_files = {
    'CDD': 'CDD_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'CFD': 'CFD_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'CWD': 'CWD_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'WW': 'WW_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'WSDI': 'WSDI_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'CSDI': 'CSDI_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc'
}

spatial_attributes_folder = '../data/spatial_attributes_netcdf/'

spatial_attribute_files = {
    'soil_types': 'soil_types.nc',
    'climate_zones': 'climate_zones.nc',
    'slope': 'slope_resampled_percent.nc',
    'elevation': 'elevation_resampled.nc',
    'irrigation': 'irrigation_aei_rf.nc'
}

In [3]:
crop = 'maize'

In [4]:
# Create crop data file and clean it
crops = ['maize']
crop_data_2005 = pd.read_csv(crop_data_2005_file,  encoding = "ISO-8859-1")
crop_data_2010 = pd.read_csv(crop_data_2010_file,  encoding = "ISO-8859-1")

merged = crop_data_2010.merge(crop_data_2005, how='inner', on='alloc_key', suffixes=['_2010', '_2005'])
kept_columns = ['alloc_key', 'x', 'y', 'iso3_2010']
for crop in crops:
    kept_columns += [f'{crop[0:4]}_a_2010', f'{crop[0:4]}_a_2005']
merged = merged[kept_columns]
for crop in crops:
    merged = merged.rename(columns={
    f'{crop[0:4]}_a_2010': f'{crop}_a_2010',
    f'{crop[0:4]}_a_2005': f'{crop}_a_2005',
})
    
merged = merged.rename(columns={
    'iso3_2010': 'iso3',
    'x': 'lon',
    'y': 'lat'
})

del crop_data_2005
del crop_data_2010
data = merged.dropna()
del merged

# Only include non zero yields
data = data[data[f'{crop}_a_2005'] > 0]
data = data[data[f'{crop}_a_2010'] > 0]
data.sort_values(['lat', 'lon'], ascending=True, inplace=True)
data

,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005
440803,15933930,147.458333,-42.708333,AUS,3917.5,3841.2
440389,15753918,146.458333,-41.208333,AUS,6138.6,6977.9
440390,15753919,146.541667,-41.208333,AUS,3926.3,4932.9
440261,15713933,147.708333,-40.875000,AUS,4496.4,4437.2
395050,15691401,-63.291667,-40.708333,ARG,5717.4,9348.4
...,...,...,...,...,...,...
170693,3980749,-117.625000,56.875000,CAN,6470.6,5886.6
170694,3980750,-117.541667,56.875000,CAN,6470.6,5886.6
170686,3970749,-117.625000,56.958333,CAN,6470.6,5886.6
170675,3830764,-116.375000,58.125000,CAN,6470.6,5886.6


In [5]:
# Compute the 10 day features
def compute_ten_day_feature(feature, file_location, year, dataset, interpolation_method='linear'):
    days = ['05', '15', '25']
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    lats_ = xr.DataArray(list(data['lat'].values), dims='z')
    lons_ = xr.DataArray(list(data['lon'].values), dims='z')
    with xr.open_dataset(file_location) as ds:
        feature_data = ds.load()
        for day in days:
            for month in months:
                time = f'{year}-{month}-{day}'
                timed = feature_data.sel(time=time).squeeze()
                feature_interpolated = timed.interp(lat=lats_, lon=lons_, method='linear')
                dataset[f'{feature}-{month}-{day}-{year}'] = getattr(feature_interpolated, feature)
    all_ten_days = []
    for month in months:
        for day in days:
            all_ten_days.append(f'{feature}-{month}-{day}-{year}')
            dataset[f'{feature}-{month}-{year}'] = dataset[f'{feature}-{month}-05-{year}'] + dataset[f'{feature}-{month}-15-{year}'] + dataset[f'{feature}-{month}-25-{year}']
    dataset = dataset.drop(columns=all_ten_days)

    return dataset

def compute_seasonal_feature(feature, file_location, year, dataset, interpolation_method='linear'):
    lats = xr.DataArray(list(dataset['lat'].values), dims='z')
    lons = xr.DataArray(list(dataset['lon'].values), dims='z')
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']
    quarter_time_mapping = {
        'Q1': '01-16',
        'Q2': '04-16',
        'Q3': '07-16',
        'Q4': '10-16'
    }
    with xr.open_dataset(file_location) as ds:
        feature_data = ds.load()
        
    for quarter in quarters:
        feature_name = f'{feature}-{quarter}-{year}'
        time = f'{year}-{quarter_time_mapping[quarter]}'
        timed = feature_data.sel(time=time).squeeze()
        feature_interpolated = timed.interp(lat=lats, lon=lons, method=interpolation_method)
        dataset[feature_name] = getattr(feature_interpolated, feature)
    return dataset

In [6]:
# Compute the seasonal features        
years = ['2010', '2009', '2006', '2005', '2004']
seasonal_features = [
    'CDD',
    'CFD',
    'CWD',
    'WW',
    'WSDI',
    'CSDI'
]

for feature in tqdm(seasonal_features):
    file_location = seasonal_data_folder + seasonal_feature_files[feature]
    for year in years:
        data = compute_seasonal_feature(feature, file_location, year, data)
data.head(10)

100%|██████████| 6/6 [00:18<00:00,  3.08s/it]


,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,CDD-Q4-2010,...,CSDI-Q3-2006,CSDI-Q4-2006,CSDI-Q1-2005,CSDI-Q2-2005,CSDI-Q3-2005,CSDI-Q4-2005,CSDI-Q1-2004,CSDI-Q2-2004,CSDI-Q3-2004,CSDI-Q4-2004
440803,15933930,147.458333,-42.708333,AUS,3917.5,3841.2,17.944444,9.687500,28.208333,6.104167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
440389,15753918,146.458333,-41.208333,AUS,6138.6,6977.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440390,15753919,146.541667,-41.208333,AUS,3926.3,4932.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440261,15713933,147.708333,-40.875000,AUS,4496.4,4437.2,28.479167,15.729167,8.937500,13.312500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
395050,15691401,-63.291667,-40.708333,ARG,5717.4,9348.4,36.027778,36.701389,33.666667,26.229167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
395051,15691402,-63.208333,-40.708333,ARG,5170.5,9332.5,37.083333,38.000000,35.395833,26.923611,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
440211,15684276,176.291667,-40.625000,NZL,11067.0,11509.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440212,15684277,176.375000,-40.625000,NZL,12313.3,12805.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440181,15674263,175.208333,-40.541667,NZL,9637.9,10023.2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440182,15674264,175.291667,-40.541667,NZL,7054.1,7336.1,8.916667,11.166667,8.000000,10.444444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Unsorted
data[data['alloc_key']== 4413637] 

,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,CDD-Q4-2010,...,WW-Q3-2010,WW-Q4-2010,WSDI-Q1-2010,WSDI-Q2-2010,WSDI-Q3-2010,WSDI-Q4-2010,CSDI-Q1-2010,CSDI-Q2-2010,CSDI-Q3-2010,CSDI-Q4-2010
9,4413637,123.041667,53.291667,CHN,3918.1,2819.2,27.0,26.326389,9.0,16.465278,...,1.0,1.083333,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0


In [16]:
data[data['alloc_key']== 4413637] 

,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,CDD-Q4-2010,...,WW-Q3-2010,WW-Q4-2010,WSDI-Q1-2010,WSDI-Q2-2010,WSDI-Q3-2010,WSDI-Q4-2010,CSDI-Q1-2010,CSDI-Q2-2010,CSDI-Q3-2010,CSDI-Q4-2010
9,4413637,123.041667,53.291667,CHN,3918.1,2819.2,27.0,26.326389,9.0,16.465278,...,1.0,1.083333,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Choose the features you want on the top of the notebook based on the ten_day_feature_files
# dict, and put them in this array.
ten_day_features = ten_day_feature_files.keys()
years = ['2010', '2009', '2006', '2005', '2004']
    
for feature in tqdm(ten_day_features):
    file_location = ten_day_data_folder + ten_day_feature_files[feature]
    for year in years:
        data = compute_ten_day_feature(feature, file_location, year, data, interpolation_method='linear')
        

data.head(10)

100%|██████████| 17/17 [08:27<00:00, 29.86s/it]


,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,CDD-Q4-2010,...,TXx-03-2004,TXx-04-2004,TXx-05-2004,TXx-06-2004,TXx-07-2004,TXx-08-2004,TXx-09-2004,TXx-10-2004,TXx-11-2004,TXx-12-2004
440803,15933930,147.458333,-42.708333,AUS,3917.5,3841.2,17.944444,9.687500,28.208333,6.104167,...,897.883992,889.619005,868.468562,862.448272,863.728408,869.762836,879.518596,893.241638,899.614735,904.204218
440389,15753918,146.458333,-41.208333,AUS,6138.6,6977.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440390,15753919,146.541667,-41.208333,AUS,3926.3,4932.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440261,15713933,147.708333,-40.875000,AUS,4496.4,4437.2,28.479167,15.729167,8.937500,13.312500,...,892.311778,892.310251,869.781061,865.159962,863.978971,868.092734,873.896744,886.515661,893.449508,893.782150
395050,15691401,-63.291667,-40.708333,ARG,5717.4,9348.4,36.027778,36.701389,33.666667,26.229167,...,923.461368,903.090675,883.332601,878.825285,878.618144,879.892602,897.409781,906.447812,911.283075,921.939113
395051,15691402,-63.208333,-40.708333,ARG,5170.5,9332.5,37.083333,38.000000,35.395833,26.923611,...,923.296304,903.158841,883.234619,878.545056,878.463872,879.741114,897.370882,906.579534,911.103692,921.852492
440211,15684276,176.291667,-40.625000,NZL,11067.0,11509.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440212,15684277,176.375000,-40.625000,NZL,12313.3,12805.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440181,15674263,175.208333,-40.541667,NZL,9637.9,10023.2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440182,15674264,175.291667,-40.541667,NZL,7054.1,7336.1,8.916667,11.166667,8.000000,10.444444,...,882.374784,881.174953,871.999545,867.423360,861.478166,865.270065,866.275394,875.059878,879.722337,888.347120


In [12]:
data.to_csv('meaned_years.csv')

In [11]:
list(data.columns)

['alloc_key',
 'lon',
 'lat',
 'iso3',
 'maize_a_2010',
 'maize_a_2005',
 'CDD-Q1-2010',
 'CDD-Q2-2010',
 'CDD-Q3-2010',
 'CDD-Q4-2010',
 'CDD-Q1-2005',
 'CDD-Q2-2005',
 'CDD-Q3-2005',
 'CDD-Q4-2005',
 'CFD-Q1-2010',
 'CFD-Q2-2010',
 'CFD-Q3-2010',
 'CFD-Q4-2010',
 'CFD-Q1-2005',
 'CFD-Q2-2005',
 'CFD-Q3-2005',
 'CFD-Q4-2005',
 'CWD-Q1-2010',
 'CWD-Q2-2010',
 'CWD-Q3-2010',
 'CWD-Q4-2010',
 'CWD-Q1-2005',
 'CWD-Q2-2005',
 'CWD-Q3-2005',
 'CWD-Q4-2005',
 'WW-Q1-2010',
 'WW-Q2-2010',
 'WW-Q3-2010',
 'WW-Q4-2010',
 'WW-Q1-2005',
 'WW-Q2-2005',
 'WW-Q3-2005',
 'WW-Q4-2005',
 'WSDI-Q1-2010',
 'WSDI-Q2-2010',
 'WSDI-Q3-2010',
 'WSDI-Q4-2010',
 'WSDI-Q1-2005',
 'WSDI-Q2-2005',
 'WSDI-Q3-2005',
 'WSDI-Q4-2005',
 'CSDI-Q1-2010',
 'CSDI-Q2-2010',
 'CSDI-Q3-2010',
 'CSDI-Q4-2010',
 'CSDI-Q1-2005',
 'CSDI-Q2-2005',
 'CSDI-Q3-2005',
 'CSDI-Q4-2005',
 'BEDD-01-2010',
 'BEDD-02-2010',
 'BEDD-03-2010',
 'BEDD-04-2010',
 'BEDD-05-2010',
 'BEDD-06-2010',
 'BEDD-07-2010',
 'BEDD-08-2010',
 'BEDD-09-2010'

In [8]:
year_bunches = [['2010', '2009'], ['2005', '2004', '2006']]
features = fs = [col.replace('-2010', '') for col in data.columns if f'-2010' in col]
for f in features:
    for b in year_bunches:
        data[f'{f}-{b[0]}'] = data[[f'{f}-{year}' for year in b]].mean(axis=1, skipna=True)
        data = data.drop(columns=[f'{f}-{year}' for year in b[1:]])
data

,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,CDD-Q4-2010,...,TXx-03-2005,TXx-04-2005,TXx-05-2005,TXx-06-2005,TXx-07-2005,TXx-08-2005,TXx-09-2005,TXx-10-2005,TXx-11-2005,TXx-12-2005
440803,15933930,147.458333,-42.708333,AUS,3917.5,3841.2,13.656250,9.677083,19.076389,6.343750,...,904.783668,889.376056,872.057050,864.060429,867.469687,875.384497,879.466295,892.615763,893.749468,905.092727
440389,15753918,146.458333,-41.208333,AUS,6138.6,6977.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440390,15753919,146.541667,-41.208333,AUS,3926.3,4932.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440261,15713933,147.708333,-40.875000,AUS,4496.4,4437.2,26.635417,13.854167,8.906250,13.156250,...,895.682178,887.488273,871.115248,866.099087,864.577964,869.347753,877.298308,885.697568,889.978545,898.319353
395050,15691401,-63.291667,-40.708333,ARG,5717.4,9348.4,28.013889,34.885417,34.569444,24.229167,...,919.693850,906.025955,885.564243,875.390753,879.072680,880.004948,893.595772,907.766975,915.929088,922.610109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170693,3980749,-117.625000,56.875000,CAN,6470.6,5886.6,17.250000,16.687500,11.562500,19.250000,...,839.252036,874.335042,886.330027,898.241606,902.367519,893.680226,882.617442,865.260804,835.210363,835.867181
170694,3980750,-117.541667,56.875000,CAN,6470.6,5886.6,17.333333,16.562500,11.687500,19.250000,...,839.591121,874.755796,886.769069,898.724645,902.792052,894.134166,883.048794,865.498656,835.421078,836.005250
170686,3970749,-117.625000,56.958333,CAN,6470.6,5886.6,17.083333,16.312500,11.437500,19.416667,...,839.620289,874.348223,886.216148,898.212108,902.207406,893.685193,882.563846,865.265549,834.456526,835.866938
170675,3830764,-116.375000,58.125000,CAN,6470.6,5886.6,21.468750,16.406250,10.593750,23.250000,...,839.206254,872.599677,885.182904,900.654757,903.980414,894.800217,881.936262,860.628922,826.967895,829.729584


In [9]:
spatial_attributes = [
    'soil_types',
    'climate_zones',
    'elevation',
    'slope',
    'irrigation'
]

categorical_spatial = [
    'soil_types',
    'climate_zones'
]

lats_ = xr.DataArray(list(data['lat'].values), dims='z')
lons_ = xr.DataArray(list(data['lon'].values), dims='z')

for feature in tqdm(spatial_attributes):
    with xr.open_rasterio(spatial_attributes_folder + spatial_attribute_files[feature]) as ds:
        feature_data = ds.load()
    if feature in categorical_spatial:
        interpolation_method = 'nearest'
    else:
        interpolation_method = 'linear'
    feature_data = feature_data.rename({'x': 'lon','y': 'lat'}).squeeze()
    data[feature] = feature_data.interp(lat=lats_, lon=lons_, method=interpolation_method)
    del feature_data

100%|██████████| 5/5 [00:00<00:00, 12.67it/s]


In [14]:
data['irrigation'].describe()

count    435996.000000
mean          5.791068
std          10.734221
min           0.000000
25%           0.937500
50%           1.229167
75%           5.604167
max          96.395833
Name: irrigation, dtype: float64

In [10]:
# Save data to a csv
file_path = f'climate_monthly_seasonal_2005_2010_soil_meaned.csv'
data.to_csv(file_path)

In [3]:
# Inspect a CSV file
file_path = f'climate_monthly_seasonal_2005_2010_soil.csv'
data = pd.read_csv(file_path)
data.head()

,Unnamed: 0,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,...,TXx-08-2005,TXx-09-2005,TXx-10-2005,TXx-11-2005,TXx-12-2005,soil_types,climate_zones,elevation,slope,irrigation
0,0,4383640,123.291667,40.0,CHN,3918.1,2819.2,27.0000,29.194444,9.048611,...,909.747803,883.116848,852.160914,809.388274,778.247625,40.0,-32768.0,-0.0,0.991271,9.0
1,2,4393629,122.375000,40.0,CHN,3119.3,2028.5,27.0000,24.604167,8.687500,...,911.883934,884.965972,854.852655,809.889451,773.909271,6.0,5.0,45.0,1.132639,3.0
2,7,4403648,123.958333,40.0,CHN,3230.1,2337.9,24.8125,26.000000,9.250000,...,909.327734,883.522990,852.087202,809.150410,780.407116,108.0,-32768.0,-0.0,0.369475,19.0
3,8,4403649,124.041667,40.0,CHN,3119.3,2010.2,23.9375,26.000000,9.250000,...,909.252123,883.555547,852.024027,809.240490,780.600624,108.0,-32768.0,-0.0,0.340288,15.0
4,9,4413637,123.041667,40.0,CHN,3918.1,2819.2,27.0000,26.326389,9.000000,...,910.897177,885.112034,853.801538,808.814855,777.211271,40.0,-32768.0,-0.0,0.991271,9.0


In [10]:
data = data.sort_values(['lon', 'lat'])
data.head()

,Unnamed: 0,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,...,TXx-08-2005,TXx-09-2005,TXx-10-2005,TXx-11-2005,TXx-12-2005,soil_types,climate_zones,elevation,slope,irrigation
411940,628376,5960690,-122.541667,4.0,USA,3651.7,3904.3,9.583333,11.750000,81.125000,...,924.540170,916.741357,905.459088,882.966962,871.803970,108.0,-32768.0,-0.0,0.000000,0.0
414001,630800,6020690,-122.541667,49.0,USA,4101.3,4385.1,10.000000,12.000000,90.291667,...,922.245411,915.581802,904.993322,883.488799,870.147676,11.0,-32768.0,-0.0,0.002703,1.0
414948,631916,6050690,-122.541667,49.0,USA,5606.8,11514.0,10.000000,12.000000,90.708333,...,921.042734,915.112926,904.377054,883.359719,868.747967,11.0,-32768.0,-0.0,0.002703,1.0
411941,628377,5960691,-122.458333,4.0,USA,4655.3,4977.4,9.416667,11.750000,79.375000,...,927.795203,919.222045,907.716976,885.054103,873.212701,108.0,-32768.0,-0.0,0.000000,0.0
412287,628785,5970691,-122.458333,4.0,USA,4993.4,5338.8,9.805556,11.916667,83.013889,...,928.433901,919.976023,908.497701,886.029521,873.940697,108.0,-32768.0,-0.0,0.000000,0.0


In [8]:
X = data[features_to_select]
y = data[f'{crop}_a_2010']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_seed)

NameError: name 'train_test_split' is not defined

In [ ]:
# Try a simple random forest regressor
regressor = RandomForestRegressor(n_estimators=200, n_jobs=10, random_state=random_seed, verbose=3)
regressor.fit(X_train, y_train)
score = regressor.score(X_test, y_test)
regressor_filename = f'{regressors_folder}/regressor_run_id_{run_id}.pkl'
pickle.dump(regressor, open(regressor_filename, 'wb'))

run_data = {
    'ten_day_features': ten_day_features,
    'seasonal_features': seasonal_features,
    'limit_change_in_yield': limit_change_in_yield,
    'limit': limit,
    'test_size': test_size,
    'sample_fraction': sample_fraction,
    'random_seed': random_seed,
    'include_coordinates': include_coordinates,
    'include_2005_maize': include_2005_maize,
    'include_seasonal': include_seasonal,
    'include_ten_day': include_ten_day,
    'include_2005_climate': include_2005_climate,
    'include_soil_type': include_soil_type,
    'include_growing_zones': include_growing_zones,
    'run_id': run_id,
    'target': target,
    'features': features,
    'read_file_path': read_file_path,
    'drop_nans': drop_nans,
    'regressor_path': regressor_filename,
    'regressor_score': score
}
with open(f'{run_folder}/run_id_{run_id}.json', 'w') as fp:
    json.dump(run_data, fp)
del X
del y
del data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import random
import re
import json
import pickle

crop = 'maize'
run_id = 42
run_folder = 'run_data'
regressors_folder = 'regressors'

with open(f'{run_folder}/run_id_{run_id}.json', 'r') as fp:
    run_data = json.load(fp)

regressor_file_name = f'{regressors_folder}/regressor_run_id_{run_id}.pkl'
regressor = pickle.load(open(regressor_file_name, 'rb'))

file_path = run_data['read_file_path']
data = pd.read_csv(file_path)

In [5]:
features_to_select = ['lon',
'maize_a_2005',
'CDD-Q1-2010',
'CDD-Q1-2005',
'CDD-Q3-2005',
'CFD-Q1-2010',
'CFD-Q4-2010',
'CFD-Q1-2005',
'CFD-Q2-2005',
'CFD-Q4-2005',
'CWD-Q2-2010',
'CWD-Q3-2010',
'CWD-Q4-2010',
'WW-Q1-2010',
'WSDI-Q3-2010',
'WSDI-Q4-2010',
'WSDI-Q2-2005',
'WSDI-Q3-2005',
'CSDI-Q1-2010',
'BEDD-06-2010',
'BEDD-10-2010',
'BEDD-04-2005',
'BEDD-06-2005',
'BEDD-08-2005',
'BEDD-09-2005',
'FD-12-2010',
'FD-02-2005',
'FD-03-2005',
'FD-05-2005',
'FD-09-2005',
'FD-11-2005',
'FD-12-2005',
'R20mm-10-2005',
'R10mm-05-2010',
'R10mm-09-2010',
'ID-03-2010',
'ID-04-2010',
'ID-11-2010',
'ID-01-2005',
'ID-02-2005',
'ID-03-2005',
'ID-04-2005',
'ID-11-2005',
'TG-03-2010',
'TG-05-2010',
'TG-07-2010',
'TG-10-2010',
'TG-12-2010',
'TG-04-2005',
'TG-05-2005',
'TG-06-2005',
'TG-07-2005',
'TG-09-2005',
'TG-12-2005',
'TN-05-2010',
'TN-06-2010',
'TN-07-2010',
'TN-09-2010',
'TN-10-2010',
'TN-05-2005',
'TN-06-2005',
'TN-09-2005',
'DTR-01-2010',
'DTR-02-2010',
'DTR-03-2010',
'DTR-07-2010',
'DTR-12-2010',
'DTR-08-2005',
'DTR-09-2005',
'DTR-11-2005',
'DTR-12-2005',
'RR1-01-2010',
'RR1-02-2010',
'RR1-03-2010',
'RR1-04-2010',
'RR1-05-2010',
'RR1-06-2010',
'RR1-09-2010',
'RR1-04-2005',
'RR1-07-2005',
'RR1-08-2005',
'RR1-09-2005',
'RR1-12-2005',
'RR-05-2010',
'RR-09-2010',
'RR-10-2010',
'RR-02-2005',
'RR-03-2005',
'RR-06-2005',
'RR-07-2005',
'RR-08-2005',
'RR-10-2005',
'RR-11-2005',
'SU-03-2010',
'SU-04-2010',
'SU-05-2010',
'SU-06-2010',
'SU-09-2010',
'SU-10-2010',
'SU-12-2010',
'SU-02-2005',
'SU-04-2005',
'SU-05-2005',
'SU-06-2005',
'SU-07-2005',
'SU-10-2005',
'SU-11-2005',
'SU-12-2005',
'TNn-05-2010',
'TNn-06-2010',
'TNn-07-2010',
'TNn-08-2010',
'TNn-09-2010',
'TNn-05-2005',
'TNn-12-2005',
'TR-06-2005',
'TR-10-2005',
'TR-12-2005',
'TX-01-2010',
'TX-02-2010',
'TX-04-2010',
'TX-08-2010',
'TX-10-2010',
'TX-02-2005',
'TX-06-2005',
'TX-07-2005',
'TX-08-2005',
'TX-09-2005',
'TX-11-2005',
'TXn-01-2010',
'TXn-02-2010',
'TXn-04-2010',
'TXn-06-2010',
'TXn-10-2010',
'TXn-12-2010',
'TXn-01-2005',
'TXn-03-2005',
'TXn-06-2005',
'TXn-09-2005',
'TXx-02-2010',
'TXx-04-2010',
'TXx-10-2010',
'TXx-11-2010',
'TXx-03-2005',
'TXx-04-2005',
'TXx-05-2005',
'TXx-06-2005',
'TXx-09-2005',
'TXx-10-2005',
"soil_types",
"climate_zones",
"elevation",
"slope",
"irrigation"
]

In [6]:
data_path = '../data/final_data.csv'
final_data = pd.read_csv(data_path)

In [11]:
final_data = final_data[abs(final_data[f'{crop}_a_2010'] - final_data[f'{crop}_a_2005']) / final_data[f'{crop}_a_2005'] < 1]

# random.shuffle(features_to_select)
X = final_data[features_to_select]


#specify the names of categorical variables 

categorical_variables = ["climate_zones",
                         "soil_types"]

#find the column indices for the categorical features

indices = [X.columns.get_loc(c) for c in categorical_variables]

categories=[]

for i in categorical_variables:
  a = sorted(list(final_data[i].unique()))
  categories.append(a)

#create the transformer
ct = ColumnTransformer([("selector", OneHotEncoder(categories=categories), indices)], remainder = 'passthrough')

#apply fit and transform
X_prepared = ct.fit_transform(X.to_numpy())


regressors_folder = 'regressors'
regressor_file_name = f'{regressors_folder}/final_model.json'
regressor = xgb.XGBRegressor(tree_method='gpu_hist')
regressor.load_model(regressor_file_name)  # load data
r2_score(regressor.predict(X_prepared), final_data['maize_a_2010'])

0.0

In [ ]:
regressor.predict(X_prepared)

In [11]:
prediction_years = [2025]
rcps = ['2p6', '4p5', '6p0', '8p5']
rcps = ['2p6']
out_file = f'../data/hadgem_predictions_year_{prediction_years}_final.csv'
regressors_folder = 'regressors'
regressor_file_name = f'{regressors_folder}/final_model.json'
regressor = xgb.Booster({'nthread': 12})  # init model
regressor.load_model(regressor_file_name)  # load data
# regressor = joblib.load(open(regressor_file_name, 'rb'))

# Drop 2010
year_2010s = [col for col in list(data.columns) if '-2010' in col]
data = data.drop(columns=year_2010s)

# Predict for different years
for rcp in tqdm(rcps):
    folder = f'../data/hadgem/hadgem_rcp_{rcp}_agroclimatic_indicators/'
    for year in tqdm(prediction_years):
        if int(year) in range(2011,2041):
            file_ending = '20110101-20401231_v1.nc'
        elif int(year) in range(2041,2071):
            file_ending = '20410101-20701231_v1.nc'
            
        for feature in seasonal_feature_files.keys():
            file_location = f'{folder}{feature}_C3S-glob-agric_hadgem2-es_rcp{rcp}_season_{file_ending}'
            data = compute_seasonal_feature(feature, file_location, str(year), data)

        for feature in ten_day_feature_files.keys():    
            file_location = f'{folder}{feature}_C3S-glob-agric_hadgem2-es_rcp{rcp}_dek_{file_ending}'
            data = compute_ten_day_feature(feature, file_location, str(year), data)

        data = data.dropna()
        year_features = [s.replace('2010', f'{year}') for s in selected_features]
        X = ct.transform(data[year_features].to_numpy())
        data[f'{rcp}_{year}_predict'] = regressor.predict(xgb.DMatrix(X))
        data = data.drop(columns=[f for f in year_features if f'-{year}' in f])
        
data.to_csv(out_file)

prediction_columns = [col for col in list(data.columns) if '_predict' in col]
cols = prediction_columns + [f'{crop}_a_2010']
predictions = data[cols]
predictions.to_csv('predictions.csv')

-136.44629678090908

In [37]:
regressor.predict(X_prepared).shape

(426309,)

In [40]:
regressor.predict(X_prepared)

array([627.85895, 561.20953, 537.45154, ..., 876.03125, 851.1409 ,
       901.8006 ], dtype=float32)

In [45]:
X = data[selected_features]
X

,lon,maize_a_2005,CDD-Q1-2010,CDD-Q1-2005,CDD-Q3-2005,CFD-Q1-2010,CFD-Q4-2010,CFD-Q1-2005,CFD-Q2-2005,CFD-Q4-2005,...,TXx-04-2005,TXx-05-2005,TXx-06-2005,TXx-09-2005,TXx-10-2005,soil_types,climate_zones,elevation,slope,irrigation
0,123.291667,2819.2,27.0000,45.229167,11.604167,90.0,68.687500,90.0,65.0,62.750000,...,861.699364,883.343974,903.371046,883.116848,852.160914,40,9,381.0,0.116577,0
1,122.375000,2028.5,27.0000,37.500000,10.729167,90.0,67.520833,90.0,65.0,64.000000,...,860.134419,884.044005,906.040748,884.965972,854.852655,40,9,518.0,1.176573,0
2,123.958333,2337.9,24.8125,50.250000,11.854167,90.0,68.250000,90.0,65.0,61.750000,...,863.072864,884.846612,902.717022,883.522990,852.087202,40,6,442.0,1.440163,0
3,124.041667,2010.2,23.9375,50.250000,11.895833,90.0,68.250000,90.0,65.0,61.750000,...,863.191762,884.927436,902.868244,883.555547,852.024027,40,6,411.0,0.371068,0
4,123.041667,2819.2,27.0000,37.208333,9.263889,90.0,72.784722,90.0,65.0,62.395833,...,861.974734,884.148724,903.710700,885.112034,853.801538,40,9,549.0,0.416216,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426304,-98.208333,6106.9,16.0000,13.250000,21.500000,2.0,0.000000,3.0,0.0,0.000000,...,928.145355,927.325173,935.745515,937.120953,926.731185,76,-997,-0.0,0.000000,1
426305,-98.125000,6107.8,16.0000,13.750000,21.500000,2.0,0.000000,3.0,0.0,0.000000,...,927.592833,926.946421,935.099796,936.687780,926.444456,76,-997,-0.0,0.000000,1
426306,-98.041667,6131.7,16.0000,14.250000,21.500000,2.0,0.000000,3.0,0.0,0.000000,...,927.040311,926.567668,934.454077,936.254608,926.157727,76,-997,-0.0,0.000000,1
426307,-97.958333,6181.2,16.0000,14.750000,21.500000,2.0,0.000000,3.0,0.0,0.000000,...,926.487789,926.188916,933.808359,935.821435,925.870998,108,-32768,-0.0,0.000000,0


In [49]:
selected_features = ['lon',
'maize_a_2005',
'CDD-Q1-2010',
'CDD-Q1-2005',
'CDD-Q3-2005',
'CFD-Q1-2010',
'CFD-Q4-2010',
'CFD-Q1-2005',
'CFD-Q2-2005',
'CFD-Q4-2005',
'CWD-Q2-2010',
'CWD-Q3-2010',
'CWD-Q4-2010',
'WW-Q1-2010',
'WSDI-Q3-2010',
'WSDI-Q4-2010',
'WSDI-Q2-2005',
'WSDI-Q3-2005',
'CSDI-Q1-2010',
'BEDD-06-2010',
'BEDD-10-2010',
'BEDD-04-2005',
'BEDD-06-2005',
'BEDD-08-2005',
'BEDD-09-2005',
'FD-12-2010',
'FD-02-2005',
'FD-03-2005',
'FD-05-2005',
'FD-09-2005',
'FD-11-2005',
'FD-12-2005',
'R20mm-10-2005',
'R10mm-05-2010',
'R10mm-09-2010',
'ID-03-2010',
'ID-04-2010',
'ID-11-2010',
'ID-01-2005',
'ID-02-2005',
'ID-03-2005',
'ID-04-2005',
'ID-11-2005',
'TG-03-2010',
'TG-05-2010',
'TG-07-2010',
'TG-10-2010',
'TG-12-2010',
'TG-04-2005',
'TG-05-2005',
'TG-06-2005',
'TG-07-2005',
'TG-09-2005',
'TG-12-2005',
'TN-05-2010',
'TN-06-2010',
'TN-07-2010',
'TN-09-2010',
'TN-10-2010',
'TN-05-2005',
'TN-06-2005',
'TN-09-2005',
'DTR-01-2010',
'DTR-02-2010',
'DTR-03-2010',
'DTR-07-2010',
'DTR-12-2010',
'DTR-08-2005',
'DTR-09-2005',
'DTR-11-2005',
'DTR-12-2005',
'RR1-01-2010',
'RR1-02-2010',
'RR1-03-2010',
'RR1-04-2010',
'RR1-05-2010',
'RR1-06-2010',
'RR1-09-2010',
'RR1-04-2005',
'RR1-07-2005',
'RR1-08-2005',
'RR1-09-2005',
'RR1-12-2005',
'RR-05-2010',
'RR-09-2010',
'RR-10-2010',
'RR-02-2005',
'RR-03-2005',
'RR-06-2005',
'RR-07-2005',
'RR-08-2005',
'RR-10-2005',
'RR-11-2005',
'SU-03-2010',
'SU-04-2010',
'SU-05-2010',
'SU-06-2010',
'SU-09-2010',
'SU-10-2010',
'SU-12-2010',
'SU-02-2005',
'SU-04-2005',
'SU-05-2005',
'SU-06-2005',
'SU-07-2005',
'SU-10-2005',
'SU-11-2005',
'SU-12-2005',
'TNn-05-2010',
'TNn-06-2010',
'TNn-07-2010',
'TNn-08-2010',
'TNn-09-2010',
'TNn-05-2005',
'TNn-12-2005',
'TR-06-2005',
'TR-10-2005',
'TR-12-2005',
'TX-01-2010',
'TX-02-2010',
'TX-04-2010',
'TX-08-2010',
'TX-10-2010',
'TX-02-2005',
'TX-06-2005',
'TX-07-2005',
'TX-08-2005',
'TX-09-2005',
'TX-11-2005',
'TXn-01-2010',
'TXn-02-2010',
'TXn-04-2010',
'TXn-06-2010',
'TXn-10-2010',
'TXn-12-2010',
'TXn-01-2005',
'TXn-03-2005',
'TXn-06-2005',
'TXn-09-2005',
'TXx-02-2010',
'TXx-04-2010',
'TXx-10-2010',
'TXx-11-2010',
'TXx-03-2005',
'TXx-04-2005',
'TXx-05-2005',
'TXx-06-2005',
'TXx-09-2005',
'TXx-10-2005',
"soil_types",
"climate_zones",
"elevation",
"slope",
"irrigation"]

len(selected_features)

154

In [ ]:
prediction_years = ['2060', '2040', '2025', '2020']
for year in tqdm(prediction_years):
    mean_included_years = [year, year + 1, year - 1]
    if year == '2040':
        mean_included_years = 
    mean_included_year_columns = [year, year + 2, year - 1]
    for rcp in rcps:
        mean_included_year_columns.append([f'{rcp}_{col}_predict' for col in mean_included_years]
        prediction[f'{year}_mean'] = prediction[mean_included_year_columns].mean(axis=1)
        prediction[f'{year}_std'] = prediction[mean_included_year_columns].std(axis=1)
                                          
predictions.to_csv(f'Predictions for Years: {years}.csv')